# 🚀 RAG_Standard 5분 퀵스타트

이 노트북에서는 RAG(Retrieval-Augmented Generation) 시스템을 5분 만에 체험합니다.

## 학습 내용
1. RAG 시스템에 질문하기
2. 검색된 문서 확인하기
3. 스트리밍 응답 체험하기

## 사전 요구사항
- RAG_Standard 서버 실행 중 (`make start`)
- 기본 URL: `http://localhost:8000`

In [ ]:
# 환경 설정
import json

import requests

# API 서버 URL 설정
# Colab 사용 시 ngrok URL로 변경하세요
BASE_URL = "http://localhost:8000"

# API Key 설정 (개발 환경에서는 비워두면 자동 스킵됨)
# 프로덕션 환경에서는 반드시 설정하세요
API_KEY = ""  # .env 파일의 FASTAPI_AUTH_KEY 값

# 공통 헤더 설정
headers = {"X-API-Key": API_KEY} if API_KEY else {}

# 연결 테스트 (health 엔드포인트는 인증 불필요)
try:
    response = requests.get(f"{BASE_URL}/health", timeout=5)
    if response.status_code == 200:
        print("✅ RAG 서버 연결 성공!")
        print(f"   상태: {response.json()['status']}")
        if not API_KEY:
            print("   ℹ️ API_KEY가 설정되지 않았습니다. 개발 환경에서만 동작합니다.")
    else:
        print(f"⚠️ 서버 응답 코드: {response.status_code}")
except requests.exceptions.ConnectionError:
    print("❌ 서버에 연결할 수 없습니다.")
    print("   'make start' 명령으로 서버를 먼저 시작하세요.")

## 1️⃣ 첫 번째 질문하기

RAG 시스템에 질문을 보내봅시다. 시스템은 관련 문서를 검색하고 AI가 답변을 생성합니다.

In [ ]:
# 채팅 API 호출
question = "RAG_Standard 어떻게 설치해?"

response = requests.post(
    f"{BASE_URL}/api/chat",
    json={"message": question},
    headers=headers,
    timeout=30
)

if response.status_code == 200:
    result = response.json()
    print(f"📝 질문: {question}")
    print(f"\n💬 답변:\n{result['answer']}")
    print(f"\n⏱️ 처리 시간: {result.get('processing_time', 'N/A')}초")
elif response.status_code == 401:
    print("❌ 인증 실패: API_KEY를 설정하세요")
    print("   .env 파일의 FASTAPI_AUTH_KEY 값을 위 셀의 API_KEY에 입력하세요")
else:
    print(f"❌ 오류: {response.status_code}")
    print(response.text)

## 2️⃣ 검색된 문서 확인

RAG는 질문과 관련된 문서를 먼저 검색합니다. 어떤 문서가 검색되었는지 확인해봅시다.

In [ ]:
# 검색된 소스 문서 표시
if response.status_code == 200:
    sources = result.get('sources', [])
    print(f"📚 검색된 문서: {len(sources)}개\n")

    for i, source in enumerate(sources, 1):
        print(f"--- 문서 {i} ---")
        print(f"📄 출처: {source.get('document', 'N/A')}")
        print(f"📊 관련도: {source.get('relevance', 0):.1%}")
        print(f"📝 미리보기: {source.get('content_preview', 'N/A')[:100]}...")
        print()
else:
    print("이전 셀을 먼저 실행하세요.")

## 3️⃣ 다양한 질문 시도

아래 예시 질문들을 시도해보세요:
- "채팅 API 사용법 알려줘"
- "환경변수 뭐 설정해야 돼?"
- "DI 컨테이너가 뭐야?"
- "테스트 어떻게 실행해?"

In [ ]:
# 직접 질문해보세요!
your_question = "채팅 API 사용법 알려줘"  # 여기를 수정하세요

response = requests.post(
    f"{BASE_URL}/api/chat",
    json={"message": your_question},
    headers=headers,
    timeout=30
)

if response.status_code == 200:
    result = response.json()
    print(f"📝 질문: {your_question}")
    print(f"\n💬 답변:\n{result['answer']}")
elif response.status_code == 401:
    print("❌ 인증 실패: 첫 번째 셀에서 API_KEY를 설정하세요")
else:
    print(f"❌ 오류: {response.text}")

## 4️⃣ 스트리밍 응답 체험

긴 답변도 실시간으로 받아볼 수 있습니다. SSE(Server-Sent Events)를 사용합니다.

In [ ]:
# 스트리밍 채팅 (SSE)
# sseclient-py 패키지가 필요합니다: pip install sseclient-py

question = "RAG 시스템의 장점을 자세히 설명해줘"

try:
    import sseclient
except ImportError:
    print("sseclient-py 패키지가 필요합니다.")
    print("설치: pip install sseclient-py")
    sseclient = None

if sseclient:
    try:
        response = requests.post(
            f"{BASE_URL}/api/chat/stream",
            json={"message": question},
            headers=headers,
            stream=True,
            timeout=60
        )

        if response.status_code == 401:
            print("❌ 인증 실패: 첫 번째 셀에서 API_KEY를 설정하세요")
        else:
            print(f"📝 질문: {question}\n")
            print("💬 답변 (실시간):")

            client = sseclient.SSEClient(response)
            for event in client.events():
                data = json.loads(event.data)
                if event.event == "chunk":
                    print(data.get("data", ""), end="", flush=True)
                elif event.event == "done":
                    print(f"\n\n✅ 완료! (총 {data.get('total_chunks', 0)}개 청크)")
                    break
                elif event.event == "error":
                    print(f"\n❌ 오류: {data.get('message', 'Unknown error')}")
                    break
    except Exception as e:
        print(f"❌ 오류: {e}")

## 🎉 축하합니다!

RAG 시스템의 기본 사용법을 익혔습니다.

### 다음 단계
- **[02_api_exploration.ipynb](02_api_exploration.ipynb)**: REST API 완전 가이드
- **[03_evaluation_demo.ipynb](03_evaluation_demo.ipynb)**: 평가 시스템 탐방

### 더 알아보기
- [API 레퍼런스](../docs/API_REFERENCE.md)
- [스트리밍 가이드](../docs/streaming-api-guide.md)
- [평가 시스템](../docs/EVALUATION_SYSTEM.md)